In [1]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from collections import Counter

from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout, Activation, Conv1D, GlobalMaxPooling1D, Input, concatenate
from keras import regularizers, initializers
from keras.optimizers import SGD, adam
from keras.callbacks import EarlyStopping, ModelCheckpoint



Using TensorFlow backend.


In [2]:
def create_dictionary(texts, vocab_size):
    """
    Creates a dictionary that maps words to ids. More frequent words have lower ids.
    The dictionary contains at the vocab_size-1 most frequent words (and a placeholder '<unk>' for unknown words).
    The place holder has the id 0.
    """
    counter = Counter()
    for tokens in texts:
        counter.update(tokens)
    vocab = [w for w, c in counter.most_common(vocab_size - 1)]
    word_to_id = {w: (i + 1) for i, w in enumerate(vocab)}
    word_to_id[UNKNOWN_TOKEN] = 0
    return word_to_id


def to_ids(words, dictionary):
    """
    Takes a list of words and converts them to ids using the word2id dictionary.
    """
    ids = []
    for word in words:
        ids.append(dictionary.get(word, dictionary[UNKNOWN_TOKEN]))
    return ids


def read_data(train_file, dev_file):
    tokenizer = TweetTokenizer()
    trainDF = pd.read_csv(train_file, sep='\t')
    devDF = pd.read_csv(dev_file, sep='\t')

    allDF = pd.concat([trainDF, devDF], ignore_index=True)
    allDF = allDF.reindex(np.random.permutation(allDF.index))
    allDF.insert(1, 'tweet_tokenized', (allDF['Tweet'].apply(lambda x: tokenizer.tokenize(x))))

    word2id = create_dictionary(allDF["tweet_tokenized"], VOCAB_SIZE)

    allDF.insert(1, 'tweet_ids', (allDF['Tweet'].apply(lambda x: to_ids(x, dictionary=word2id))))

    allDF['all'] = allDF.iloc[:, -11:].values.tolist()

    total = len(allDF)
    trainend = int(total * 0.8)
    devend = trainend + int(total * 0.1)
    return allDF.iloc[:trainend, :], allDF.iloc[trainend:devend, :], allDF.iloc[devend:, :]


def evaluate(predictions, y_test):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    all_correct = 0
    for i, pred in enumerate(predictions):
        for j, em in enumerate(pred):
            if em >= 0.5:
                if y_test[i][j] == 1:
                    tp += 1
                else:
                    fp += 1
            if em < 0.5:
                if y_test[i][j] == 1:
                    fn += 1
                else:
                    tn += 1
            if tp + tn == y_test.shape[1]:
                all_correct += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    print("F1: {}\nPrecision: {}\nRecall: {}\nCompletely correct: {}".format(f1, precision, recall, all_correct))


In [3]:
data_dir = 'data/'
train_file = os.path.join(data_dir, '2018-E-c-En-train.txt')
dev_file = os.path.join(data_dir, '2018-E-c-En-dev.txt')

VOCAB_SIZE = 100000
MAX_LEN = 100
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 50
EPOCHS = 10  # Standard 10
UNKNOWN_TOKEN = "<unk>"
EMOTIONS = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
            'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

In [4]:
trainDF, devDF, testDF = read_data(train_file, dev_file)
x_train = sequence.pad_sequences(np.array(trainDF['tweet_ids']), maxlen=MAX_LEN)
x_dev = sequence.pad_sequences(np.array(devDF['tweet_ids']), maxlen=MAX_LEN)
x_test = sequence.pad_sequences(np.array(testDF['tweet_ids']), maxlen=MAX_LEN)
y_train = np.array([trainDF['all']])[0]
y_dev = np.array([devDF['all']])[0]
y_test = np.array([testDF['all']])[0]


In [57]:
main_input = Input(shape=(MAX_LEN,), dtype='int32', name='main_input')
x = Embedding(VOCAB_SIZE, EMBEDDING_SIZE)(main_input)
x = Conv1D(
    filters=2 * HIDDEN_SIZE,
    kernel_size=3,
    activation='tanh',
    strides=1,
    padding='valid', )(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.5)(x)
predictions = Dense(11, activation='sigmoid')(x)



In [58]:
# main_input = Input(shape=(MAX_LEN,), dtype='int32', name='main_input')
# x = Embedding(VOCAB_SIZE, EMBEDDING_SIZE)(main_input)
# x = Bidirectional(LSTM(EMBEDDING_SIZE))(x)
# # x = GlobalMaxPooling1D()(x)
# # x = Dropout(0.5)(x)
# predictions = Dense(11, activation='sigmoid')(x)


In [59]:

model = Model(inputs=main_input, outputs=predictions)

In [60]:
# opt = adam(lr=0.01)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'],

              )


In [61]:

early_stopper = EarlyStopping(monitor='val_acc', patience=5, mode='max')
checkpoint = ModelCheckpoint(data_dir+'model.m', save_best_only=True, monitor='val_acc', mode='max')

In [62]:
def compute_class_weights(y_train):
    class_totals = np.sum(y_train, axis=0)
    overall_sum = np.sum(y_train)
    class_weights = [((overall_sum - i) / overall_sum) ** 10 for i in class_totals]

    return class_weights


class_weights = compute_class_weights(y_train)


In [63]:

model.fit(
    x_train,
    y_train,
    # batch_size=BATCH_SIZE,
    callbacks=[early_stopper, checkpoint,],
    epochs=EPOCHS,
    validation_data=(x_dev, y_dev),
    class_weight=class_weights,
    verbose=1
)

Train on 6179 samples, validate on 772 samples
Epoch 1/10


  32/6179 [..............................] - ETA: 3:16 - loss: 0.6920 - acc: 0.5142

  96/6179 [..............................] - ETA: 1:10 - loss: 0.6713 - acc: 0.6411

 160/6179 [..............................] - ETA: 44s - loss: 0.6565 - acc: 0.6875 

 224/6179 [>.............................] - ETA: 34s - loss: 0.6458 - acc: 0.7094

...

5952/6179 [===========================>..] - ETA: 0s - loss: 0.4122 - acc: 0.8191

6016/6179 [============================>.] - ETA: 0s - loss: 0.4123 - acc: 0.8191

6080/6179 [============================>.] - ETA: 0s - loss: 0.4124 - acc: 0.8189

6144/6179 [============================>.] - ETA: 0s - loss: 0.4121 - acc: 0.8191

6179/6179 [==============================] - 10s 2ms/step - loss: 0.4118 - acc: 0.8192 - val_loss: 0.4265 - val_acc: 0.8115


In [64]:
best_model = load_model(data_dir+'model.m')

In [65]:
predictions = best_model.predict(x_test)


In [66]:
evaluate(predictions, y_test)


F1: 0.4052074857607811
Precision: 0.751131221719457
Recall: 0.27743732590529246
Completely correct: 1


In [ ]:
#TODO: kommentieren, Parameter testen, Code sauber machen, Test Ergebnisse notieren